In [1]:
import automation1 as a1
import time
import numpy as np
import matplotlib.pyplot as plt

import os
import serial
import time

import math

In [26]:
## make sure is available on Saw1 computer
from pathlib import Path
from pprint import pprint

# Connect to Controller

In [3]:
controller = a1.Controller.connect()
controller.start()
print(controller.is_running)

True


# A. Setup & Constants

In [4]:
base_path = Path(r"C:\Users\UNIVERSITY\Desktop\RunData\Automation1_TEST\SpindleC\CutCammingThin")
# Filenames
cuttype     = "Thin"                # Equivalent to #define cuttype "Thin"
mastername  = "Master.txt"          # The list of cuts
datafile    = "DataCollectCut.dat"  # Not needed? Data collection output
outfile     = "MetCheck.txt"        # Not needed? Metrology check output
lockname    = "lockfile.lock"       # Lockfile to prevent re-running

In [5]:
# Parameters
safelift        = 10.0   # 20 mm above z-start for safe moves
numpoints       = 10     # NOT NEEDED number of points (if doing metrology checks)
lifter_settle_s = 2    # seconds
feedspeed       = 5.0   # 11 mm/s 

# Build full file paths (Path objects)
master_path = base_path / mastername
data_path   = base_path / datafile
out_path    = base_path / outfile
lock_path   = base_path / lockname

# Just to verify in notebook
print("Base path: ", base_path)
print("Master file: ", master_path)
print("Lock file: ", lock_path)

Base path:  C:\Users\UNIVERSITY\Desktop\RunData\Automation1_TEST\SpindleC\CutCammingThin
Master file:  C:\Users\UNIVERSITY\Desktop\RunData\Automation1_TEST\SpindleC\CutCammingThin\Master.txt
Lock file:  C:\Users\UNIVERSITY\Desktop\RunData\Automation1_TEST\SpindleC\CutCammingThin\lockfile.lock


# B. Initial Motions, Digital Output (DO) (Turn Spindle/Flood Cooling On), lockfile check

- This section, in the order that it currently exists, works well.
- Notice that it involves a lot of pauses and resumes and looks for the attributes
- here, i can isolate step B to be its own separate script for now---knowing that it will do the setup of enabling the relevant axes, move them up to 0 in Z, and turn on the spindle and flood cooling appropriately for ZC at the moment

In [ ]:
lock_path = base_path / lockname
if lock_path.exists():
    print(f"[guard] Lockfile present at {lock_path} — stopping here.")
    ### Need to actually add a stopping component if putting this in script form ###

**To do**: need a way to query state of task window before running command queue to ensure exception errors in python don't occur

In [ ]:
# start a queue with capacity 64, and block if full
cq = controller.runtime.commands.begin_command_queue(task=1, command_capacity=64, should_block_if_full=True)
print("Queue started on task:", cq.task_index, "capacity:", cq.command_capacity)


In [ ]:
cq.pause()

In [ ]:
# Enable axes
for axis in ["X", "Y", "ZC"]:
    cq.commands.motion.enable(axis)

In [ ]:
from pprint import pprint

s = cq.status  # property, not callable

# Show non-callable public attributes + their values
attrs = {
    name: getattr(s, name)
    for name in dir(s)
    if not name.startswith("_") and not callable(getattr(s, name, None))
}
pprint(attrs)


In [ ]:
cq.resume()

In [ ]:
cq.pause()

In [ ]:
# queued versions of the same calls you used before
#cq.commands.motion.moveabsolute(["ZA"], [0.0],     [20.0])
#cq.commands.motion.moveabsolute(["ZB"], [0.0],     [20.0])
cq.commands.motion.moveabsolute(["ZC"], [-0.0005], [20.0])
cq.commands.motion.waitforinposition(["ZC"])


In [ ]:
from pprint import pprint

s = cq.status  # property, not callable

# Show non-callable public attributes + their values
attrs = {
    name: getattr(s, name)
    for name in dir(s)
    if not name.startswith("_") and not callable(getattr(s, name, None))
}
pprint(attrs)


In [ ]:
cq.resume()

In [ ]:
from pprint import pprint

s = cq.status  # property, not callable

# Show non-callable public attributes + their values
attrs = {
    name: getattr(s, name)
    for name in dir(s)
    if not name.startswith("_") and not callable(getattr(s, name, None))
}
pprint(attrs)


In [ ]:
cq.pause()

In [ ]:
## turn on the digital outputs and set the dwell in the queue
io_axis_misc  = "X"  # adjust if your IO lives elsewhere
io_axis_flood = "X"

cq.commands.io.digitaloutputset(axis=io_axis_misc,  output_num=10, value=1)
cq.commands.io.digitaloutputset(axis=io_axis_flood, output_num=6,  value=1)
cq.commands.motion.movedelay(["X","Y","ZC"], delay_time=11_000)  # 11 s


In [ ]:
cq.resume()   
cq.wait_for_empty()  # block until all queued commands above finish


In [ ]:
from pprint import pprint

s = cq.status  # property, not callable

# Show non-callable public attributes + their values
attrs = {
    name: getattr(s, name)
    for name in dir(s)
    if not name.startswith("_") and not callable(getattr(s, name, None))
}
pprint(attrs)


In [ ]:
controller.runtime.commands.end_command_queue(cq)
## we're done with the command queue as we query some states before restarting it but this leaves us with 
## spindle C on and flood cooling on even after command queue is ended

#### ZC Drive Status Check 

In [ ]:
cfg = a1.StatusItemConfiguration()
cfg.axis.add(a1.AxisStatusItem.DriveStatus,     "ZC")
cfg.axis.add(a1.AxisStatusItem.ProgramPosition, "ZC")

results = controller.runtime.status.get_status_items(cfg)

# Extract values
drive_status = results.axis.get(a1.AxisStatusItem.DriveStatus, "ZC").value
program_pos  = results.axis.get(a1.AxisStatusItem.ProgramPosition, "ZC").value

# Convert drive status to int and mask off camming bit (bit 16)
camming_bit = int(drive_status) & (1 << 16)

print(f"[diag] ZC camming bit set? {'Yes' if camming_bit else 'No'}")
print(f"[diag] ZC ProgramPosition = {program_pos:.4f} mm")


# C. Open Master.txt File
- can confirm this does proper checking of Master.txt and camming files
- good to 'finalize'

In [6]:
assert master_path.exists(), f"Master file not found: {master_path}"
print("[step C] Found Master.txt:", master_path)

with open(master_path, "r") as f:
    lines = [ln.strip() for ln in f if ln.strip()]

print(f"[step C] Read {len(lines)} line(s) from Master.txt.")
if lines:
    print(" first line →", lines[0])

[step C] Found Master.txt: C:\Users\UNIVERSITY\Desktop\RunData\Automation1_TEST\SpindleC\CutCammingThin\Master.txt
[step C] Read 5 line(s) from Master.txt.
 first line → 1770 177.986 -172.66421914434062 -23.0098180322404 -117.66421914434062


In [7]:
# parse the first cut definition
# AeroBasic FILEREAD expected 5 numeric values per row
# camnum  xvalue  ystart  zstart  yend
row0 = lines[0].split()
assert len(row0) >= 5, f"Expected ≥5 fields, got {len(row0)}: {row0}"

camnum  = int(row0[0])
xvalue  = float(row0[1])
ystart  = float(row0[2])
zstart  = float(row0[3])
yend    = float(row0[4])

print(f"[step C] camnum={camnum}, x={xvalue}, ystart={ystart}, zstart={zstart}, yend={yend}")

# these two mirror variables the AeroBasic computed (useful later if we do met checks)
#yval = ystart
#dy   = (yend - ystart) / numpoints if numpoints else 0.0


#print(f"[step C] yval={yval}, dy={dy}")


[step C] camnum=1770, x=177.986, ystart=-172.66421914434062, zstart=-23.0098180322404, yend=-117.66421914434062


In [8]:
## build the cam table file name for this row and check it exists
cam_filename = f"CutCam{cuttype}{camnum:04d}.Cam"
cam_path     = base_path / cam_filename

print("[step C] Cam file:", cam_path)
if not cam_path.exists():
    print("Cam file not found.")



[step C] Cam file: C:\Users\UNIVERSITY\Desktop\RunData\Automation1_TEST\SpindleC\CutCammingThin\CutCamThin1770.Cam


# D. Main Loop over Master.txt Rows

In [9]:
assert cam_path.exists(), f"Cam file not found: {cam_path}"
print("[step D] Using cam file:", cam_path)

with open(cam_path, "r") as f:
    shown = 0
    for ln in f:
        s = ln.strip()
        if not s or s.startswith(("#",";")):
            continue
        print(" cam>", s)
        shown += 1
        if shown >= 5:
            break

[step D] Using cam file: C:\Users\UNIVERSITY\Desktop\RunData\Automation1_TEST\SpindleC\CutCammingThin\CutCamThin1770.Cam
 cam> Number of points 0290
 cam> Master Units (PRIMARY)
 cam> Slave Units (PRIMARY)
 cam> 0001 -172.66421914434062 -23.0098180322404
 cam> 0002 -172.16421914434062 -23.01000955316378


# Sanah made changes here on August 21, 2025

In [14]:
leader_values = []
follower_values = []

with open(cam_path, "r") as f:
    for ln in f:
        s = ln.strip()
        parts = s.replace(",", " ").split()
        if len(parts) < 2:
            continue
        try:
            leader = float(parts[1]) # was 0 
            follower = float(parts[2]) # was 1
        except ValueError:
            print(f"[warn] Skipping non-numeric line: {s}")
            continue
        leader_values.append(leader)
        follower_values.append(follower)


[warn] Skipping non-numeric line: ;Filename: C:/Users/UNIVERSITY/Desktop/RunData/CCAT_Alumina_350/Surface1/0deg/SpindleC/CutCammingThin/CutCamThin1770.Cam
[warn] Skipping non-numeric line: Number of points 0290
[warn] Skipping non-numeric line: Master Units (PRIMARY)
[warn] Skipping non-numeric line: Slave Units (PRIMARY)


In [16]:
print(leader_values[0:10])
print(follower_values[0:10])

[-172.66421914434062, -172.16421914434062, -171.66421914434062, -171.16421914434062, -170.66421914434062, -170.16421914434062, -169.66421914434062, -169.16421914434062, -168.66421914434062, -168.16421914434062]
[-23.0098180322404, -23.01000955316378, -23.0102013266822, -23.0103933663813, -23.01058568556081, -23.0107782972293, -23.01097121409896, -23.01116444858074, -23.01135801277947, -23.01155191848908]


In [17]:
# start a queue with capacity 64, and block if full
cq = controller.runtime.commands.begin_command_queue(task=1, command_capacity=64, should_block_if_full=True)
print("Queue started on task:", cq.task_index, "capacity:", cq.command_capacity)


Queue started on task: 1 capacity: 64


In [18]:
####### initiate advanced motion module ############
am = cq.commands.advanced_motion

In [19]:
# free table 1 in case something is already loaded
am.cammingfreetable(1) # table_num = 1

In [20]:
am.cammingloadtablefromarray(
    table_num=1,
    leader_values=leader_values,
    follower_values=follower_values,
    num_values=len(leader_values),
    units_mode=a1.CammingUnits.Primary,               # follower is position vs leader
    interpolation_mode=a1.CammingInterpolation.Linear, # typical for .Cam
    wrap_mode=a1.CammingWrapping.NoWrap,               # NOWRAP
    table_offset=0.0,
#    execution_task_index=1
)
print("[step D] Camming table 1 loaded.")

[step D] Camming table 1 loaded.


In [ ]:
## good to check the above table

In [21]:
# slow, safe test speeds
SPEED_Y_TRAVERSE  = 20.0   # mm/s
SPEED_X_TRAVERSE  = 15.0   # mm/s
SPEED_ZC_APPROACH = 2.0   # mm/s  (down to zstart+2)
SPEED_ZC_TOUCH    = 0.5   # mm/s  (final settle at zstart)


In [22]:
cq.pause()

In [23]:
ystart

-172.66421914434062

In [24]:
# move y to start position, wait for in position
cq.commands.motion.moveabsolute(["Y"], [ystart], [SPEED_Y_TRAVERSE])
cq.commands.motion.waitforinposition(["Y"])
print("queued Y→ystart + wait.")

queued Y→ystart + wait.


In [27]:
s = cq.status  # property, not callable

# Show non-callable public attributes + their values
attrs = {
    name: getattr(s, name)
    for name in dir(s)
    if not name.startswith("_") and not callable(getattr(s, name, None))
}
pprint(attrs)


{'is_empty': False,
 'is_paused': True,
 'number_of_executed_commands': 2,
 'number_of_times_emptied': 2,
 'number_of_unexecuted_commands': 2}


In [28]:
cq.resume()

In [29]:
xvalue

177.986

In [30]:
## move X 
cq.commands.motion.moveabsolute(["X"], [xvalue], [SPEED_X_TRAVERSE])
print("[D-Q4] queued X→xvalue.")

[D-Q4] queued X→xvalue.


In [31]:
s = cq.status  # property, not callable

# Show non-callable public attributes + their values
attrs = {
    name: getattr(s, name)
    for name in dir(s)
    if not name.startswith("_") and not callable(getattr(s, name, None))
}
pprint(attrs)


{'is_empty': True,
 'is_paused': False,
 'number_of_executed_commands': 5,
 'number_of_times_emptied': 4,
 'number_of_unexecuted_commands': 0}


In [ ]:
cq.pause()

In [32]:
## move ZC --- do we need to do this? yes
cq.commands.motion.moveabsolute(["ZC"], [zstart + 2.0], [SPEED_ZC_APPROACH])
cq.commands.motion.moveabsolute(["ZC"], [zstart],        [SPEED_ZC_TOUCH])
cq.commands.motion.waitforinposition(["ZC"])

In [36]:
s = cq.status  # property, not callable

# Show non-callable public attributes + their values
attrs = {
    name: getattr(s, name)
    for name in dir(s)
    if not name.startswith("_") and not callable(getattr(s, name, None))
}
pprint(attrs)


{'is_empty': True,
 'is_paused': False,
 'number_of_executed_commands': 9,
 'number_of_times_emptied': 6,
 'number_of_unexecuted_commands': 0}


In [34]:


cq.commands.advanced_motion.cammingon(
    follower_axis="ZC",
    leader_axis="Y",
    table_num=1,
    source=a1.CammingSource.PositionCommand,                    # leader uses position
    output=a1.CammingOutput.RelativePosition  # matches CAMSYNC ...,1
)
print("[D-Q6] queued camming ON (ZC follows Y, relative).")

[D-Q6] queued camming ON (ZC follows Y, relative).


CHECK ZC AXIS STATUS FOR CAMMING BIT STATE
-- meaning follower camming mode for ZC is engaged and ZC is ready to follow the camming table as soon as Y advances

In [37]:
cfg = a1.StatusItemConfiguration()
cfg.axis.add(a1.AxisStatusItem.AxisStatus,        "ZC")  # bitfield we need
cfg.axis.add(a1.AxisStatusItem.ProgramPosition,   "ZC")  # optional: handy context

results = controller.runtime.status.get_status_items(cfg)


# Pull raw values (note the .value, same as the doc’s .Value)
axis_status_item = results.axis.get(a1.AxisStatusItem.AxisStatus, "ZC")
progpos_item     = results.axis.get(a1.AxisStatusItem.ProgramPosition, "ZC")

axis_status = int(axis_status_item.value)   # bitfield
zc_progpos  = float(progpos_item.value)     # mm (primary user units)

CAMMING_MASK = 1 << 16  # AeroBasic INDEXTOMASK(16) == 65536
is_camming   = bool(axis_status & CAMMING_MASK)

print(f"[diag] ZC AxisStatus = 0x{axis_status:08X}")
print(f"[diag] ZC camming bit set? {'Yes' if is_camming else 'No'}")
print(f"[diag] ZC ProgramPosition = {zc_progpos:.4f}")


[diag] ZC AxisStatus = 0x00012009
[diag] ZC camming bit set? Yes
[diag] ZC ProgramPosition = -23.0098


In [ ]:
#### once camming bit set is True, we're right where the Aerobasic command would say CAMSYNC. so next steps is to check the spindle 

In [39]:
controller.runtime.commands.end_command_queue(cq)

In [41]:
## checking if spindle speed is 0
## MAKE SURE THE COMMAND QUEUE IS ENDED BEFORE RUNNING THIS. THROWS TASK STATE ERROR IF COMMAND QUEUE IS STILL ON
io_axis_spindle   = "X"   # axis/module hosting the DI bank
spindle_input_num = 0     # from AeroBasic: DI[0]

val = controller.runtime.commands.io.digitalinputget(
    axis=io_axis_spindle, input_num=spindle_input_num, execution_task_index=1
)
print(f"[spindle] {io_axis_spindle}.DI[{spindle_input_num}] = {val}  (1 ⇒ Spindle Speed 0)")

if val == 1:
    raise RuntimeError("Spindle Speed 0 — aborting before lead-in.")


[spindle] X.DI[0] = 0  (1 ⇒ Spindle Speed 0)


In [52]:
# start a queue with capacity 64, and block if full
cq = controller.runtime.commands.begin_command_queue(task=1, command_capacity=64, should_block_if_full=True)
print("Queue started on task:", cq.task_index, "capacity:", cq.command_capacity)


Queue started on task: 1 capacity: 64


In [54]:
# lead-in: Y from ystart → ystart + 15 mm (slow)
cq.commands.motion.moveabsolute(["ZC"], [0], [8])


In [ ]:
cq.pause()

In [43]:
s = cq.status  # property, not callable

# Show non-callable public attributes + their values
attrs = {
    name: getattr(s, name)
    for name in dir(s)
    if not name.startswith("_") and not callable(getattr(s, name, None))
}
pprint(attrs)


{'is_empty': True,
 'is_paused': False,
 'number_of_executed_commands': 0,
 'number_of_times_emptied': 0,
 'number_of_unexecuted_commands': 0}


In [44]:
### slow test speeds for cut camming right now

LEAD_IN_DIST     = 15.0   # mm
LEAD_IN_SPEED    = 4.0    # mm/s
FEED_SPEED_TEST  = 7.0    # mm/s
SPEED_ZC_RETRACT = 2.0    # mm/s


In [45]:
# lead-in: Y from ystart → ystart + 15 mm (slow)
cq.commands.motion.moveabsolute(["Y"], [ystart + LEAD_IN_DIST], [LEAD_IN_SPEED])


In [ ]:
s = cq.status  # property, not callable

# Show non-callable public attributes + their values
attrs = {
    name: getattr(s, name)
    for name in dir(s)
    if not name.startswith("_") and not callable(getattr(s, name, None))
}
pprint(attrs)


In [ ]:
cq.resume()

In [46]:
cfg = a1.StatusItemConfiguration()
cfg.axis.add(a1.AxisStatusItem.AxisStatus,        "ZC")  # bitfield we need
cfg.axis.add(a1.AxisStatusItem.ProgramPosition,   "ZC")  # optional: handy context

results = controller.runtime.status.get_status_items(cfg)


# Pull raw values (note the .value, same as the doc’s .Value)
axis_status_item = results.axis.get(a1.AxisStatusItem.AxisStatus, "ZC")
progpos_item     = results.axis.get(a1.AxisStatusItem.ProgramPosition, "ZC")

axis_status = int(axis_status_item.value)   # bitfield
zc_progpos  = float(progpos_item.value)     # mm (primary user units)

CAMMING_MASK = 1 << 16  # AeroBasic INDEXTOMASK(16) == 65536
is_camming   = bool(axis_status & CAMMING_MASK)

print(f"[diag] ZC AxisStatus = 0x{axis_status:08X}")
print(f"[diag] ZC camming bit set? {'Yes' if is_camming else 'No'}")
print(f"[diag] ZC ProgramPosition = {zc_progpos:.4f}")

[diag] ZC AxisStatus = 0x00012009
[diag] ZC camming bit set? Yes
[diag] ZC ProgramPosition = -23.0098


In [47]:
# cut feed: Y to yend (slow test feed)
cq.commands.motion.moveabsolute(["Y"], [yend], [FEED_SPEED_TEST])
cq.commands.motion.waitforinposition(["Y"])


In [ ]:
# small, safe test move while camming is ON
TEST_DY = 20.0   # mm (keep small)
SPEED_Y_TEST = 3.0

cfg = a1.StatusItemConfiguration()
cfg.axis.add(a1.AxisStatusItem.ProgramPosition, "Y")
y_start = float(controller.runtime.status.get_status_items(cfg).axis.get(a1.AxisStatusItem.ProgramPosition, "Y").value)
y_target = y_start + TEST_DY
print(f"[probe] Y: {y_start:.4f} → {y_target:.4f}")

In [ ]:
y_target

In [ ]:
# queue the move (use your existing queue or start a short-lived one)
cq_probe = controller.runtime.commands.begin_command_queue(task=1, command_capacity=64, should_block_if_full=True)


In [ ]:
cq_probe.commands.motion.moveabsolute(["Y"], [y_target], [SPEED_Y_TEST])

In [ ]:
cfg = a1.StatusItemConfiguration()
cfg.axis.add(a1.AxisStatusItem.ProgramPosition, "Y")
cfg.axis.add(a1.AxisStatusItem.ProgramPosition, "ZC")

samples = []
t0 = time.time()
timeout_s = 5.0

while True:
    res = controller.runtime.status.get_status_items(cfg)
    y  = float(res.axis.get(a1.AxisStatusItem.ProgramPosition, "Y").value)
    zc = float(res.axis.get(a1.AxisStatusItem.ProgramPosition, "ZC").value)
    samples.append((time.time() - t0, y, zc))

    # exit when we reach (or pass) target or timeout
    if y >= y_target - 1e-3:
        break
    if (time.time() - t0) > timeout_s:
        print("[probe] timeout")
        break
    time.sleep(0.02)  # ~50 Hz polling

# drain & end the probe queue
cq_probe.wait_for_empty()
controller.runtime.commands.end_command_queue(cq_probe)

# compute deltas and an empirical slope dZC/dY
y0, z0 = samples[0][1], samples[0][2]
y1, z1 = samples[-1][1], samples[-1][2]
dY = y1 - y0
dZ = z1 - z0

# robust slope estimate using all samples (simple least-squares on deltas)
dy_list = []
dz_list = []
for k in range(1, len(samples)):
    dy = samples[k][1] - samples[0][1]
    dz = samples[k][2] - samples[0][2]
    dy_list.append(dy)
    dz_list.append(dz)
slope = (dz_list[-1] / dy_list[-1]) if dy_list[-1] != 0 else float('nan')

print(f"[probe] ΔY = {dY:.4f}, ΔZC = {dZ:.4f}")
print(f"[probe] empirical slope dZC/dY ≈ {slope:.4f} (user units/user units)")

# Let's test some ZC camming situations

In [ ]:
pwd

In [ ]:
# 1) load a tiny, obvious table on an unused table number (e.g., 9)
leader_synth   = [0.0, 10.0]   # Y +10
follower_synth = [0.0,  5.0]   # ZC +5 (clear 0.5 slope)

am = controller.runtime.commands.advanced_motion
am.cammingfreetable(9)
am.cammingloadtablefromarray(
    table_num=9,
    leader_values=leader_synth,
    follower_values=follower_synth,
    num_values=len(leader_synth),
    units_mode=a1.CammingUnits.Primary,
    interpolation_mode=a1.CammingInterpolation.Linear,
    wrap_mode=a1.CammingWrapping.NoWrap,
    table_offset=0.0
)

In [ ]:
# 2) make sure ZC is camming OFF, then turn camming ON (non-queued) using table 9
am.cammingoff(follower_axis="ZC")  # safe even if already off

In [ ]:
am.cammingon(
    follower_axis="ZC",
    leader_axis="Y",
    table_num=9,
    source=a1.CammingSource.PositionCommand,
    output=a1.CammingOutput.RelativePosition
)

In [ ]:
# 3) move Y by +4 and read ΔZC
motion = controller.runtime.commands.motion


In [ ]:
# capture ZC start
cfg = a1.StatusItemConfiguration()
cfg.axis.add(a1.AxisStatusItem.ProgramPosition, "ZC")
zc0 = float(controller.runtime.status.get_status_items(cfg).axis.get(a1.AxisStatusItem.ProgramPosition, "ZC").value)

# move Y a bit (within the 0..10 domain)
cfgY = a1.StatusItemConfiguration(); cfgY.axis.add(a1.AxisStatusItem.ProgramPosition, "Y")
y0 = float(controller.runtime.status.get_status_items(cfgY).axis.get(a1.AxisStatusItem.ProgramPosition, "Y").value)
motion.moveabsolute(["Y"], [y0 + 4.0], [2.0])
motion.waitforinposition(["Y"])

zc1 = float(controller.runtime.status.get_status_items(cfg).axis.get(a1.AxisStatusItem.ProgramPosition, "ZC").value)
print(f"[synth] ΔZC ≈ {zc1 - zc0:.4f} (expected ≈ +2.0)")

In [ ]:
# cam OFF (AeroBasic: SYNC ZC 1 0)
cq.commands.advanced_motion.cammingoff(follower_axis="ZC")


In [ ]:
# retract ZC and free table 1
cq.commands.moveabsolute(["ZC"], [zstart + safelift], [SPEED_ZC_RETRACT])
cq.commands.waitforinposition(["ZC"])
cq.commands.advanced_motion.cammingfreetable(1)


In [ ]:
# drain the queued work
cq.wait_for_empty()
print("[post-cut] lead-in, cut, cam OFF, ZC retract, free table — done.")

In [ ]:
## verify camming is off
drive_status = int(results.axis.get(a1.AxisStatusItem.DriveStatus, "ZC"))
camming_bit  = bool(drive_status & (1 << 16))
print(f"[diag] ZC camming bit after cammingoff? {camming_bit}")

In [ ]:
# Keep or create a global/current row index i
######## haven't created a loop or a function yet because just testing raw steps ########
######## at the moment to make sure everything makes sense first #######
try:
    i
except NameError:
    i = 0  # if not defined yet, start at first row already used

i += 1  # advance to next row
if i >= len(raw_lines):
    print("[next] No more cuts — reached end of Master.txt")
else:
    row = raw_lines[i].split()
    assert len(row) >= 5, f"Expected 5 fields on line {i}, got {len(row)}: {row}"

    camnum  = int(row[0])
    xvalue  = float(row[1])
    ystart  = float(row[2])
    zstart  = float(row[3])
    yend    = float(row[4])

    cam_filename = f"CutCam{cuttype}{camnum:04d}.Cam"mm
    cam_path     = base_path / cam_filename

    print(f"[next] i={i} camnum={camnum}  x={xvalue}  ystart={ystart}  zstart={zstart}  yend={yend}")
    print(f"[next] cam file → {cam_path}")
    # Now repeat Step D: parse cam file → load table → stage → cammingon → spindle check → lead-in & cut → cammingoff → retract → free.

# Cuts are Done from Master.txt File

In [ ]:
# drain and end any open queues on task 1
for qname in ("cq"): #, "cq2"):
    try:
        q = globals().get(qname)
        if q is not None:
            try:
                q.wait_for_empty()
            except Exception:
                pass
            controller.runtime.commands.end_command_queue(q)
            print(f"[E] ended queue: {qname}")
    except Exception as e:
        print(f"[E] (note) could not end {qname}: {e}")

In [ ]:
# flood coolant off (D06=0)
io_axis_flood = "X"  # change if your flood IO lives elsewhere
controller.commands.io.digitaloutputset(axis=io_axis_flood, output_num=6, value=0, execution_task_index=1)
print("flood coolant OFF (DO6=0).")


In [ ]:
### no command_queue okay??
motion = controller.commands.motion
motion.moveabsolute(["ZC"], [-0.0005], [20.0], execution_task_index=1)
motion.waitforinposition(["ZC"], execution_task_index=1)
print("[E] ZC parked at -0.0005.")

In [ ]:
### create lockfile
import datetime as _dt
lock_path = base_path / lockname
msg = f"The cutting has been completed, directory is now locked ({_dt.datetime.now().isoformat(timespec='seconds')})"

with open(lock_path, "w", encoding="utf-8") as f:
    f.write(msg + "\n")

print(f"[E] wrote lockfile → {lock_path}")

In [ ]:
###### still going to leave spindle running by the way

# End the Queue

In [55]:
controller.runtime.commands.end_command_queue(cq)

# Command Queue Python Module

WARNINGS FROM THE DOCUMENTATION:
- must keep command queue populated at all times. If the command queue is not populated, your process will stall and motion problems might occur
- if a `MovePt()` or `MovePvt()` command is the most recently executed command from the queue, and a starvation of the command queue has occurred, the controlloer **will not** automatically decelerate the axes that you specified to a comand to zero velocity 

NOTES FROM THE DOCUMENTATION
- when the command queue begins on a task, the controller executes all the commands that are in the command queue as quickly as possible. If the controller automatically executes commands from the command queue more quickly than you can add them, the command queue will not have a sufficient quantity of commands to execute. This condition is known as a starvation of the command queue. 
- while the command queue is active, you can examine its status to find the number of times that a starvation of the command queue has occurred. 
- if starvation mode of the queue occurs, velocity blending mode is enabled on the task, AND most recently executed command from the command queue is a `MoveCcw()`, `MoveCw()`, or `MoveLinear()` command then the controller automatically decelerates them to zero velocity to prevent motion problems from occurring 
- can use `CommandQueueCount` task status item to make the first command wait for the command queue to fill with a specified number of commands before the controller executes them.

RELEVANT LINK: 
- http://help.aerotech.com/automation1/Content/APIs/Python/References/Command-Queue-Python.htm?Highlight=advanced_motion

# Status Command Queue Commands to Have in Arsenal

In [ ]:
command_queue.status.number_of_times.emptied

In [ ]:
command_queue.execute("wait(StatusGetTaskItem ... >= 20)")

# Important thoughts to factor in
1. Make sure UI threading is not going to cause problems with motion. Be careful in understanding this deeply. https://help.aerotech.com/automation1/Content/APIs/Python/Get-Started/Guidelines-Python.htm
2. add a stopping command execution component if lockfile present. right now, it's just a print statement.
3. Two ways to use the queue (both OK):
“Stream & drain”: enqueue → cq.wait_for_empty() → enqueue more → … → end_command_queue(cq).
“Arm & go”: cq.pause() → enqueue everything → cq.resume() → cq.wait_for_empty() → end_command_queue(cq).
4. have we lost the feedspeed anywhere in this code? No
5. need a way to query state of task window before running command queue to ensure exception errors in python don't occur
6. is safelift being factored into the code (was it used in aeroscript source---i would think so?)
7. before turnning camming bit off, we need to check the state of ZC camming as part of a safety check in the code for robustness